<a href="https://colab.research.google.com/github/tantanbei/deep-learning/blob/main/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E4%B9%8B%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%85%A5%E9%97%A8%E6%9C%80%E4%BD%B3%E8%B7%AF%E5%BE%84_%E5%A4%9A%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print("TensorFlow version:", tf.__version__)
import _pickle as cPickle
import numpy as np
import os


CIFAR_DIR = '/content/cifar-10-batches-py'

Instructions for updating:
non-resource variables are not supported in the long term


TensorFlow version: 2.15.0


In [ ]:
#@title 数据cifar-10准备
print(os.getcwd())

#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class.
#There are 50000 training images and 10000 test images.
#下载
os.system("wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz")

#解压
os.system("tar -zxvf cifar-10-python.tar.gz")

CIFAR_DIR = '/content/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

/content
['readme.html', 'data_batch_2', 'batches.meta', 'test_batch', 'data_batch_4', 'data_batch_3', 'data_batch_5', 'data_batch_1']


In [ ]:
# @title  搭建tensorflow计算图

tf.reset_default_graph()

# 搭建tensorflow计算图
x = tf.placeholder(tf.float32, [None, 3072])     # None表示可变性

# [None]
y = tf.placeholder(tf.int64, [None])

# (3072, 10)
w = tf.get_variable('w', [x.get_shape()[-1], 10],   # 因为是十分类，所以只有一个输出结果，定义为10
                   initializer=tf.random_normal_initializer(0, 1))   #initializer表示初始化，这里使用正态分布,均值为0，方差为1

# (10, )
b = tf.get_variable('b', [10],
                   initializer=tf.constant_initializer(0.0))   # b使用常量初始化
# get_variable表示获取变量，如果已经定义好了就使用，如果没有就定义


# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b

# mean square loss
'''
# 因为是多分类，使用softmax
# course:1+e^x
# api:e^x/sum(e^x)
# 归一化
# p_y:每个样本都是一个分布，10个值加起来等于1，[[0.01,0.9,...0.03], []...]
p_y = tf.nn.softmax(y_)

# 多分类的loss，可以使用one-hot编码把y也变成一个分布
# 5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)

loss = tf.reduce_mean(tf.square(y_one_hot - p_y))   #  类型不一致需要变换
'''

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> one_hot
# loss = ylogy_

# 得到的是index，是一个int值
predict = tf.argmax(y_, 1)

# [1, 0, 1, 1, 0, 1,...]
correct_prediction = tf.equal(predict, y)   # bool

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 定义梯度下降的方法
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)    # 最小化loss，到这里，计算图构建完成

In [ ]:
# 为了使函数具有更好的泛化能力，定义一个load_data
# 用于从pickle文件将数据读取进来
def load_data(filename):
    '''read data from data file.'''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# CIFAR-10数据处理的方法

class CifarData:
    # 初始化：filenames,文件名说明对于训练数据集和测试数据集是分开的
    # need_shuffle:对于训练集来说，在训练的时候，过完一遍数据之后，需要shuffle一下数据，就是使得数据更加散乱，散乱代表数据之间没有依赖关系，使得泛化能力更强，这是机器学习中一个常用的概念
    # 对于测试集来说，因为不需要训练，所以不需要shuffle
    def __init__(self, filenames, need_shuffle):
        # 把数据读进来
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)

        # 把最后的值合并并且转化为numpy的矩阵
        self._data = np.vstack(all_data)     # 从纵向上把数据合并到一起，因为all_data定义是一个一个向量
        self._data = self._data / 127.5 - 1    # 一般来说，我们会把图像缩放到-1到1之间，所以进行缩放：归一化
        self._labels = np.hstack(all_labels)  # 从横向上合并到一起，因为label实际上是一个一维向量

        # 测试
        print(self._data.shape)
        print(self._labels.shape)

        self._num_examples = self._data.shape[0]  # 数量
        self._need_shuffle = need_shuffle   # shuffle开关
        self._indicator = 0   # 表示训练集已经遍历到哪个位置了
        if self._need_shuffle:
            self._shuffle_data()

    # 因为all_data和all_labels都是numpy的数据结构，所以我们可以用numpy的方法来做
    def _shuffle_data(self):
        # np.random.permutation这个函数会做一个混排，从0到_num_examples，如[0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0, 1]
        p = np.random.permutation(self._num_examples)
        # p只是得到一个排列，需要把数据放进来
        # _data和_labels应该做一样的shuffle，否则会错乱
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self, batch_size):
        '''return batch_size examples as a batch.'''
        end_indicator = self._indicator + batch_size  # 对应于_indicator，这个表示结束位置
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')

        # 当end_indicator=batch_size时，如果还比_num_examples大，就需要抛出异常了
        if end_indicator > self._num_examples:
            raise Exception('batch size is larger than all examples')

        batch_data = self._data[self._indicator: end_indicator]   # 需要把这个范围内的数据放到结果中
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)

(50000, 3072)
(50000,)


In [ ]:
# 执行计算图，首先要初始化变量
init = tf.global_variables_initializer()

batch_size = 200
train_steps = 100000    # 因为是一个神经元，所以还是挺快的
test_steps = 10  # 20 * 200 = 2000 张图片

# 对于tensorflow来说，在构建好计算图后，需要开启一个会话session
# 打开会话之后，就可以执行计算图了
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)

        # 在会话中执行计算图使用sess.run()方法，如果加了train_op,就说明在这次run中我们就去训练了，如果没有，就是在测试模式下
        # feed_dict:要喂入的数据，因为前面x和y都是placeholder，所以需要输入数据，x和y应该输入的是CIFAR-10的数据，分别是图片数据和label数据
        # 需要在CIFAR-10数据集上循环遍历整个数据，使得feed_dict拿到的是不同的数据
        # 为了达到这种处理模式，我们需要对CIFAR-10的数据集做一些处理
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={x: batch_data, y: batch_labels})

        # 打印log
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))

        if (i+1) % 5000 == 0:
            # 因为test不用shuffle，所以遍历完之后抛出异常，所以需要在遍历完一次之后重新创建CifarData类
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []    # 需要做一个总的test上的结果，所以需要把这些结果加起来做平均
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)

            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

            # 两次测试准确率是一样的，说明没有学习到什么东西，我们需要做一下改进——》CifarData

[Train] Step: 500, loss: 13.63286, acc: 0.21500
[Train] Step: 1000, loss: 9.54653, acc: 0.23000
[Train] Step: 1500, loss: 8.34347, acc: 0.22500
[Train] Step: 2000, loss: 6.96906, acc: 0.30500
[Train] Step: 2500, loss: 6.92920, acc: 0.26500
[Train] Step: 3000, loss: 6.00010, acc: 0.27500
[Train] Step: 3500, loss: 5.12461, acc: 0.32000
[Train] Step: 4000, loss: 5.20169, acc: 0.26500
[Train] Step: 4500, loss: 4.70586, acc: 0.27000
[Train] Step: 5000, loss: 4.82999, acc: 0.29500
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.27200
[Train] Step: 5500, loss: 4.22627, acc: 0.27500
[Train] Step: 6000, loss: 4.22771, acc: 0.33500
[Train] Step: 6500, loss: 3.88664, acc: 0.31500
[Train] Step: 7000, loss: 3.48683, acc: 0.36500
[Train] Step: 7500, loss: 3.65555, acc: 0.35500
[Train] Step: 8000, loss: 3.64383, acc: 0.29500
[Train] Step: 8500, loss: 3.93059, acc: 0.31000
[Train] Step: 9000, loss: 3.22177, acc: 0.30500
[Train] Step: 9500, loss: 3.20751, acc: 0.35500
[Train] Step: 10000, loss: 3.3305